In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
import pickle

In [3]:
df=pd.read_csv('diabetes.csv')

In [4]:
df_orig=pd.read_csv('diabetes.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Glucose  768 non-null    int64  
 1   Insulin  768 non-null    int64  
 2   BMI      768 non-null    float64
 3   Age      768 non-null    int64  
 4   Outcome  768 non-null    int64  
dtypes: float64(1), int64(4)
memory usage: 30.1 KB


In [6]:
df.head()

,Glucose,Insulin,BMI,Age,Outcome
0,148,0,33.6,50,1
1,85,0,26.6,31,0
2,183,0,23.3,32,1
3,89,94,28.1,21,0
4,137,168,43.1,33,1


In [7]:
## OUTLIER DETECTION AND REMOVAL : NOT ASKED SO NOT DONE !!
# # Calculate Q1 and Q3
# Q1 = df[['Glucose', 'Insulin', 'BMI', 'Age']].quantile(0.25)
# Q3 = df[['Glucose', 'Insulin', 'BMI', 'Age']].quantile(0.75)
# IQR = Q3 - Q1

# # Identify outliers
# outliers = ((df[['Glucose', 'Insulin', 'BMI', 'Age']] < (Q1 - 1.5 * IQR)) |
#             (df[['Glucose', 'Insulin', 'BMI', 'Age']] > (Q3 + 1.5 * IQR)))

# print("Outliers:\n", outliers.sum())


In [8]:
# lower_bound = Q1 - 1.5 * IQR
# upper_bound = Q3 + 1.5 * IQR
# df = df[~((df[['Glucose', 'Insulin', 'BMI', 'Age']] < lower_bound) |
#                   (df[['Glucose', 'Insulin', 'BMI', 'Age']] > upper_bound)).any(axis=1)]

In [9]:
# Standard Scaling due to varied ranges of different attributes in given dataset
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_features = scaler.fit_transform(df[['Glucose', 'Insulin', 'BMI', 'Age']])
df = pd.DataFrame(scaled_features, columns=['Glucose', 'Insulin', 'BMI', 'Age'])
df.head()

,Glucose,Insulin,BMI,Age
0,0.848324,-0.692891,0.204013,1.425995
1,-1.123396,-0.692891,-0.684422,-0.190672
2,1.943724,-0.692891,-1.103255,-0.105584
3,-0.998208,0.123302,-0.494043,-1.041549
4,0.504055,0.765836,1.409746,-0.020496


In [10]:
from sklearn.model_selection import train_test_split
df = pd.concat([df, df_orig['Outcome']], axis=1)
X = df[['Glucose', 'Insulin', 'BMI', 'Age']]
y = df['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training set size:", X_train.shape)
print("Testing set size:", X_test.shape)


Training set size: (614, 4)
Testing set size: (154, 4)


In [11]:
# Gaussian Naive-Bayes Classifier
naive_bayes_model = GaussianNB()
naive_bayes_model.fit(X_train, y_train)
y_pred_nb = naive_bayes_model.predict(X_test)

In [12]:
# Perceptron Model
perceptron_model = Perceptron(max_iter=1000, tol=1e-3, random_state=42)
perceptron_model.fit(X_train, y_train)
y_pred_perceptron = perceptron_model.predict(X_test)

In [13]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

accuracy_nb = accuracy_score(y_test, y_pred_nb)
conf_matrix_nb = confusion_matrix(y_test, y_pred_nb)
class_report_nb = classification_report(y_test, y_pred_nb)

print("Naive Bayes Model Accuracy:", accuracy_nb)
print("Confusion Matrix:\n", conf_matrix_nb)
print("Classification Report:\n", class_report_nb)

accuracy_perceptron = accuracy_score(y_test, y_pred_perceptron)
conf_matrix_perceptron = confusion_matrix(y_test, y_pred_perceptron)
class_report_perceptron = classification_report(y_test, y_pred_perceptron)

print("Perceptron Model Accuracy:", accuracy_perceptron)
print("Confusion Matrix:\n", conf_matrix_perceptron)
print("Classification Report:\n", class_report_perceptron)

Naive Bayes Model Accuracy: 0.7467532467532467
Confusion Matrix:
 [[80 19]
 [20 35]]
Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.81      0.80        99
           1       0.65      0.64      0.64        55

    accuracy                           0.75       154
   macro avg       0.72      0.72      0.72       154
weighted avg       0.75      0.75      0.75       154

Perceptron Model Accuracy: 0.564935064935065
Confusion Matrix:
 [[66 33]
 [34 21]]
Classification Report:
               precision    recall  f1-score   support

           0       0.66      0.67      0.66        99
           1       0.39      0.38      0.39        55

    accuracy                           0.56       154
   macro avg       0.52      0.52      0.52       154
weighted avg       0.56      0.56      0.56       154



In [ ]:
#TODO : Highlight Strengths and Weaknesses

In [14]:
with open('naive_bayes_model.pkl', 'wb') as nb_file:
    pickle.dump(naive_bayes_model, nb_file)
with open('perceptron_model.pkl', 'wb') as perc_file:
    pickle.dump(perceptron_model, perc_file)
print("Models saved successfully.")

Models saved successfully.


In [ ]:
##LOADING MODELS LATER
# # Load the Naive Bayes model
# with open('naive_bayes_model.pkl', 'rb') as nb_file:
#     loaded_naive_bayes_model = pickle.load(nb_file)

# # Load the Perceptron model
# with open('perceptron_model.pkl', 'rb') as perc_file:
#     loaded_perceptron_model = pickle.load(perc_file)

# print("Models loaded successfully.")

In [17]:
pip install Flask Flask-CORS